In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

# 4 important portfolio statistic
- cummulative returns
  - (last value / beginning value) - 1
- average daily return
  - average the data
- standard [deviation] daily returns (or risk or volatility)
  - compute standard deviation
- sharpe ratio
  - consider reward in the context of risk
  - (portfolio returns - risk free return) / std dev. of portfolio return

Note: daily returns on first day is always 0 and therefore should be excluded from calculation in CR, avg daily returns, standard daily return and sharpe ratio

# Sharpe ratio:

- video 07:

S = mean(daily_returns - daily_risk_free_return) / std(daily_returns)

- to calculate Sharpe ratio, we will need daily_returns

daily_risk_free_return = 0

## Sampling and sharpe ratio:

- sharpe ratio vary widely depending on how frequently you sample
- original intent was to sample it per year; so here is how to get annualized Sharpe Ratio:
`SRannualized = k * S` where  `k = sqrt(# samples per year)`

- if using `daily return` k = sqrt(252); for `weekly` k = sqrt(52); for `monthly` k= sqrt(12)


## What is sharpe ratio given the following?

- 60 days of data
- average daily return = 10 bps = 0.0010
- daily risk free rate = 2 bps = 0.0002
- standard deviation of daily returns = 10 bps = 0.0010
<hr />

- Step 1: numerator 10 -2 = 8bps
- Step 2: denumerator = 10 bps
- Step 3: numerator / denumerator = 0.8
- Step 4: convert to annualized since data data is based on daily data: sqrt(252) * 0.8 = 12.7


# Try code from lecture for  portfolio value

In [2]:
def get_data(symbols, dates):
    df = DataFrame(index=dates)
    if 'SPY' not in symbols:  # add SPY for reference, if absent
        symbols.insert(0, 'SPY')
    for symbol in symbols:
        df_temp = pd.read_csv(f"data/{symbol}.csv", index_col='Date',
            parse_dates=True, usecols=['Date', 'Adj Close'], na_values=['nan']
            ).rename(columns={'Adj Close': symbol})
        df = df.join(df_temp)
        if symbol == 'SPY':
            df = df.dropna(subset=["SPY"])
    return df

def plot_data(df, title="Stock prices", ylabel="Price"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(title=title, fontsize=12)
    ax.set_xlabel("Date")
    ax.set_ylabel(ylabel)
    plt.grid(True)
    plt.show()

def print_head_tail(data, size=3):
    print(data.head(size))
    print("----------------------------------")
    print(data.tail(size))


In [3]:
start_val = 1000000
start_date = '2009-01-01'
end_date = '2011-12-31'
symbols = ['SPY', 'XOM', 'GOOG', 'GLD']
allocs = [0.4, 0.4, 0.1, 0.1]
prices = get_data(symbols, pd.date_range(start_date, end_date))

In [4]:
print_head_tail(prices)

              SPY    XOM    GOOG    GLD
2009-01-02  86.80  74.48  321.32  86.23
2009-01-05  86.70  74.47  328.05  84.48
2009-01-06  87.28  73.26  334.06  85.13
----------------------------------
               SPY    XOM   GOOG     GLD
2011-12-28  123.64  82.61  639.7  151.03
2011-12-29  124.92  83.68  642.4  150.34
2011-12-30  124.31  83.18  645.9  151.99


## Now calculate the value of portfolio day by day with out starting sum till the end date

In [5]:
initial_prices = prices.iloc[0, :]
prices_normed = prices / initial_prices # makes initial day price 1 & everything else relative to it
print_head_tail(prices_normed)
# every row shows price at each day; remember inital day price was $1

                 SPY       XOM      GOOG       GLD
2009-01-02  1.000000  1.000000  1.000000  1.000000
2009-01-05  0.998848  0.999866  1.020945  0.979705
2009-01-06  1.005530  0.983620  1.039649  0.987243
----------------------------------
                 SPY       XOM      GOOG       GLD
2011-12-28  1.424424  1.109157  1.990850  1.751479
2011-12-29  1.439171  1.123523  1.999253  1.743477
2011-12-30  1.432143  1.116810  2.010146  1.762612


In [6]:
# multiply normed prices and allocations of our equity
alloced = prices_normed * allocs
print_head_tail(alloced)

                 SPY       XOM      GOOG       GLD
2009-01-02  0.400000  0.400000  0.100000  0.100000
2009-01-05  0.399539  0.399946  0.102094  0.097971
2009-01-06  0.402212  0.393448  0.103965  0.098724
----------------------------------
                 SPY       XOM      GOOG       GLD
2011-12-28  0.569770  0.443663  0.199085  0.175148
2011-12-29  0.575668  0.449409  0.199925  0.174348
2011-12-30  0.572857  0.446724  0.201015  0.176261


In [7]:
# from allocation to dollar amount allocated
# every row shows the value of assets at each day
pos_vals = alloced * start_val
print_head_tail(pos_vals)

                      SPY            XOM           GOOG            GLD
2009-01-02  400000.000000  400000.000000  100000.000000  100000.000000
2009-01-05  399539.170507  399946.294307  102094.485248   97970.543894
2009-01-06  402211.981567  393447.905478  103964.894809   98724.341876
----------------------------------
                      SPY            XOM           GOOG            GLD
2011-12-28  569769.585253  443662.728249  199085.024275  175147.860373
2011-12-29  575668.202765  449409.237379  199925.308104  174347.674823
2011-12-30  572857.142857  446723.952739  201014.564920  176261.162009


In [8]:
port_val = pos_vals.sum(axis=1)  # portfolio value at the end of day for each day
print_head_tail(port_val.round(4))

# last row shows our total value

2009-01-02    1000000.0000
2009-01-05     999550.4940
2009-01-06     998349.1237
dtype: float64
----------------------------------
2011-12-28    1.387665e+06
2011-12-29    1.399350e+06
2011-12-30    1.396857e+06
dtype: float64


# Alternatively can do this, using just allocation

In [9]:
print_head_tail(alloced)

                 SPY       XOM      GOOG       GLD
2009-01-02  0.400000  0.400000  0.100000  0.100000
2009-01-05  0.399539  0.399946  0.102094  0.097971
2009-01-06  0.402212  0.393448  0.103965  0.098724
----------------------------------
                 SPY       XOM      GOOG       GLD
2011-12-28  0.569770  0.443663  0.199085  0.175148
2011-12-29  0.575668  0.449409  0.199925  0.174348
2011-12-30  0.572857  0.446724  0.201015  0.176261


In [10]:
alloced_vals = alloced.sum(axis=1)
print_head_tail(alloced_vals.round(4))

2009-01-02    1.0000
2009-01-05    0.9996
2009-01-06    0.9983
dtype: float64
----------------------------------
2011-12-28    1.3877
2011-12-29    1.3994
2011-12-30    1.3969
dtype: float64


In [11]:
daily_portfolio_val = alloced_vals * start_val
print_head_tail(daily_portfolio_val.round(4))

2009-01-02    1000000.0000
2009-01-05     999550.4940
2009-01-06     998349.1237
dtype: float64
----------------------------------
2011-12-28    1.387665e+06
2011-12-29    1.399350e+06
2011-12-30    1.396857e+06
dtype: float64


In [12]:
def compute_daily_returns(df): # see lecture 1-4
    daily_returns = df.copy()
    price_t = df[1:]
    price_t_minus_1 = df[:-1].values # so that values are not aligned by index and elementwise ops is doable
    daily_returns[1:] = (price_t / price_t_minus_1) - 1
    daily_returns.iloc[0] = 0
    return daily_returns[1:]

## Lets write a function to do all this!
def get_sharpe_ratio(daily_portfolio_val):
    daily_return = compute_daily_returns(daily_portfolio_val)
    mean_daily_port_val = daily_return.mean()
    sd_daily_port_val = daily_return.std()
    k = np.sqrt(252) # 252 for daily frequency
    daily_sharpe = mean_daily_port_val/sd_daily_port_val
    print(f"Mean: {mean_daily_port_val} and SD: {sd_daily_port_val}")
    annualized_sharpe = k * daily_sharpe
    return annualized_sharpe

def get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, allocs):
    # all above code in one spot!
    prices = get_data(symbols, pd.date_range(start_date, end_date))
    initial_prices = prices.iloc[0, :]
    prices_normed = prices / initial_prices # makes initial day price 1 & everything else relative to it
    alloced = prices_normed * allocs
    daily_alloced_vals = alloced.sum(axis=1)
    daily_portfolio_val = daily_alloced_vals * start_val 
    
    # lets add sharpe ratio too
    sr = get_sharpe_ratio(daily_portfolio_val)
    print(f"Start port val: ${daily_portfolio_val.iloc[0].round(2)}; End port val ${daily_portfolio_val.iloc[-1].round(2)}. \nSharpe Ratio is {sr} for allocation {allocs}")
    print("  ")
    return daily_portfolio_val, sr


In [13]:
start_val = 1000000
start_date = '2009-01-01'
end_date = '2011-12-31'
symbols = ['SPY', 'XOM', 'GOOG', 'GLD']

_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.4, 0.4, 0.1, 0.1])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.91, 0.03, 0.03, 0.03])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.2, 0.5, 0.15, 0.15])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.1, 0.1, 0.7, 0.1])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.03, 0.03, 0.91, 0.03])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.01, 0.01, 0.5642, 0.4158])

# Notice that higher return doesn't necessarily mean higher sharpe ratio
# between second from last and the last one, 
#  Second from last return = $195.... & SR = 0.94
#  Last one: return = $189... but SR is 1.21!

Mean: 0.0005164937963338702 and SD: 0.012143025495758337
Start port val: $1000000.0; End port val $1396856.82. 
Sharpe Ratio is 0.6752093895962253 for allocation [0.4, 0.4, 0.1, 0.1]
  
Mean: 0.0005863484946687713 and SD: 0.013722928836652899
Start port val: $1000000.0; End port val $1449937.01. 
Sharpe Ratio is 0.6782804095145463 for allocation [0.91, 0.03, 0.03, 0.03]
  
Mean: 0.0005222420956421482 and SD: 0.011522062436952075
Start port val: $1000000.0; End port val $1410747.1. 
Sharpe Ratio is 0.7195184283017161 for allocation [0.2, 0.5, 0.15, 0.15]
  
Mean: 0.0009225573509385479 and SD: 0.015252690254036153
Start port val: $1000000.0; End port val $1838258.39. 
Sharpe Ratio is 0.9601679232155291 for allocation [0.1, 0.1, 0.7, 0.1]
  
Mean: 0.0010419763144892784 and SD: 0.017441628839572506
Start port val: $1000000.0; End port val $1958579.47. 
Sharpe Ratio is 0.9483553028843703 for allocation [0.03, 0.03, 0.91, 0.03]
  
Mean: 0.0009170326637706207 and SD: 0.011998945386412753
Star

## Sharpe ratio note:

A higher sharpe ratio implies a better return, a lower volatility, or both. There’s no guarantee that a higher sharpe ratio means a better cumulative return.

In [15]:
# Sharpie ratio is not affected by $$$ of investment!
start_val = 1

_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.4, 0.4, 0.1, 0.1])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.91, 0.03, 0.03, 0.03])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.2, 0.5, 0.15, 0.15])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.1, 0.1, 0.7, 0.1])
_ = get_daily_port_val_and_sharpe_ratio(start_val, start_date, end_date, symbols, [0.03, 0.03, 0.91, 0.03])


Mean: 0.0005164937963338709 and SD: 0.012143025495758332
Start port val: $1.0; End port val $1.4. 
Sharpe Ratio is 0.6752093895962266 for allocation [0.4, 0.4, 0.1, 0.1]
  
Mean: 0.0005863484946687728 and SD: 0.013722928836652897
Start port val: $1.0; End port val $1.45. 
Sharpe Ratio is 0.6782804095145482 for allocation [0.91, 0.03, 0.03, 0.03]
  
Mean: 0.0005222420956421444 and SD: 0.011522062436952071
Start port val: $1.0; End port val $1.41. 
Sharpe Ratio is 0.7195184283017111 for allocation [0.2, 0.5, 0.15, 0.15]
  
Mean: 0.0009225573509385451 and SD: 0.015252690254036154
Start port val: $1.0; End port val $1.84. 
Sharpe Ratio is 0.960167923215526 for allocation [0.1, 0.1, 0.7, 0.1]
  
Mean: 0.0010419763144892795 and SD: 0.017441628839572506
Start port val: $1.0; End port val $1.96. 
Sharpe Ratio is 0.9483553028843713 for allocation [0.03, 0.03, 0.91, 0.03]
  
